In [3]:
from __future__ import (division, print_function)
# import time

import networkx as nx
import pickle

import pandas as pd
import random
# import matplotlib.pyplot as plt
# from scipy import stats
# from scipy.spatial import distance

import torch
import torch.utils.data
import torch.optim as optim
import torch.nn.functional as F

from arg_helper import *
from model import *
# from args import *
# from data import *
# from data_parallel import *
from evaluation import *
import os
import argparse

import torch
import torchvision as tv
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.nn.functional as F

import networkx as nx
import pickle as pkl
import scipy.sparse as sp
import logging

import numpy as np
import random
from random import shuffle

import shutil
import os
import time
from model import *
from utils import *

# Set seeds
random.seed(10)

def bfs_seq(G, start_id):
    '''
    get a bfs node sequence
    :param G:
    :param start_id:
    :return:
    '''
    dictionary = dict(nx.bfs_successors(G, start_id))
    start = [start_id]
    output = [start_id]
    while len(start) > 0:
        next = []
        while len(start) > 0:
            current = start.pop(0)
            neighbor = dictionary.get(current)
            if neighbor is not None:
                #### a wrong example, should not permute here!
                # shuffle(neighbor)
                next = next + neighbor
        output = output + next
        start = next
    return output



def encode_adj(adj, max_prev_node=10, is_full = False):
    '''

    :param adj: n*n, rows means time step, while columns are input dimension
    :param max_degree: we want to keep row number, but truncate column numbers
    :return:
    '''
    if is_full:
        max_prev_node = adj.shape[0]-1

    # pick up lower tri
    adj = np.tril(adj, k=-1)
    n = adj.shape[0]
    adj = adj[1:n, 0:n-1]

    # use max_prev_node to truncate
    # note: now adj is a (n-1)*(n-1) matrix
    adj_output = np.zeros((adj.shape[0], max_prev_node))
    for i in range(adj.shape[0]):
        input_start = max(0, i - max_prev_node + 1)
        input_end = i + 1
        output_start = max_prev_node + input_start - input_end
        output_end = max_prev_node
        adj_output[i, output_start:output_end] = adj[i, input_start:input_end]
        adj_output[i,:] = adj_output[i,:][::-1] # reverse order

    return adj_output

def encode_adj_flexible(adj):
    '''
    return a flexible length of output
    note that here there is no loss when encoding/decoding an adj matrix
    :param adj: adj matrix
    :return:
    '''
    # pick up lower tri
    adj = np.tril(adj, k=-1)
    n = adj.shape[0]
    adj = adj[1:n, 0:n-1]

    adj_output = []
    input_start = 0
    for i in range(adj.shape[0]):
        input_end = i + 1
        adj_slice = adj[i, input_start:input_end]
        adj_output.append(adj_slice)
        non_zero = np.nonzero(adj_slice)[0]
        input_start = input_end-len(adj_slice)+np.amin(non_zero)

    return adj_output


########## use pytorch dataloader
class Graph_sequence_sampler_pytorch(torch.utils.data.Dataset):
    def __init__(self, G_list, max_num_node=None, max_prev_node=None, iteration=20000):
        self.adj_all = []
        self.len_all = []
        for G in G_list:
            self.adj_all.append(np.asarray(nx.to_numpy_matrix(G)))
            self.len_all.append(G.number_of_nodes())
        if max_num_node is None:
            self.n = max(self.len_all)
        else:
            self.n = max_num_node
        if max_prev_node is None:
            print('calculating max previous node, total iteration: {}'.format(iteration))
            self.max_prev_node = max(self.calc_max_prev_node(iter=iteration))
            print('max previous node: {}'.format(self.max_prev_node))
        else:
            self.max_prev_node = max_prev_node

        # self.max_prev_node = max_prev_node

        # # sort Graph in descending order
        # len_batch_order = np.argsort(np.array(self.len_all))[::-1]
        # self.len_all = [self.len_all[i] for i in len_batch_order]
        # self.adj_all = [self.adj_all[i] for i in len_batch_order]
    def __len__(self):
        return len(self.adj_all)
    def __getitem__(self, idx):
        adj_copy = self.adj_all[idx].copy()
        x_batch = np.zeros((self.n, self.max_prev_node))  # here zeros are padded for small graph
        x_batch[0,:] = 1 # the first input token is all ones
        y_batch = np.zeros((self.n, self.max_prev_node))  # here zeros are padded for small graph
        # generate input x, y pairs
        len_batch = adj_copy.shape[0]
        x_idx = np.random.permutation(adj_copy.shape[0])
        adj_copy = adj_copy[np.ix_(x_idx, x_idx)]
        adj_copy_matrix = np.asmatrix(adj_copy)
        G = nx.from_numpy_matrix(adj_copy_matrix)
        # then do bfs in the permuted G
        start_idx = np.random.randint(adj_copy.shape[0])
        x_idx = np.array(bfs_seq(G, start_idx))
        adj_copy = adj_copy[np.ix_(x_idx, x_idx)]
        adj_encoded = encode_adj(adj_copy.copy(), max_prev_node=self.max_prev_node)
        # get x and y and adj
        # for small graph the rest are zero padded
        y_batch[0:adj_encoded.shape[0], :] = adj_encoded
        x_batch[1:adj_encoded.shape[0] + 1, :] = adj_encoded
        return {'x':x_batch, 'y':y_batch, 'len':len_batch, 'max_prev_node': self.max_prev_node}

    def calc_max_prev_node(self, iter=20000,topk=10):
        max_prev_node = []
        for i in range(iter):
            if i % (iter / 5) == 0:
                print('iter {} times'.format(i))
            adj_idx = np.random.randint(len(self.adj_all))
            adj_copy = self.adj_all[adj_idx].copy()
            # print('Graph size', adj_copy.shape[0])
            x_idx = np.random.permutation(adj_copy.shape[0])
            adj_copy = adj_copy[np.ix_(x_idx, x_idx)]
            adj_copy_matrix = np.asmatrix(adj_copy)
            G = nx.from_numpy_matrix(adj_copy_matrix)
            # then do bfs in the permuted G
            start_idx = np.random.randint(adj_copy.shape[0])
            x_idx = np.array(bfs_seq(G, start_idx))
            adj_copy = adj_copy[np.ix_(x_idx, x_idx)]
            # encode adj
            adj_encoded = encode_adj_flexible(adj_copy.copy())
            max_encoded_len = max([len(adj_encoded[i]) for i in range(len(adj_encoded))])
            max_prev_node.append(max_encoded_len)
        max_prev_node = sorted(max_prev_node)[-1*topk:]
        return max_prev_node

In [4]:
graphs_whole = pd.read_pickle("ZeoGraphs.p")
unit_cell_whole = pd.read_pickle("ZeoUnitCells.p")
max_num_nodes = 432
batch_size = 100
batch_share = 20
max_num_nodes_l = 3
max_num_nodes_g = 120
print("Zeo data is being used")

Zeo data is being used


In [25]:
class Make_batch_data:
    def __init__(self, num_pad, batch_size, batch_share):
        """
        num_pad: padding size
        """
        self.batch_size = batch_size
        self.batch_share = batch_share
        self.num_pad = num_pad
    
    def makeAdj(self, graphs, num_pad):
        adj = torch.zeros(len(graphs), num_pad, num_pad)
        for i in range(len(graphs)):
            graph_tmp = torch.from_numpy(nx.to_numpy_array(graphs[i]))
            adj[i, :, :] = F.pad(graph_tmp, (0, num_pad - graph_tmp.shape[1], 0, num_pad - graph_tmp.shape[0]), "constant", 0)
        
        return adj
    
    def makeTrain(self, dataset, unit_cell, num_per_unit_cell):
        '''
        dataset: list of networkx graphs
        unit_cell: Numpy array of unit cells identifier
        '''
        # Number of graphs
        num_graph = len(dataset)
        
        print("Number of graphs:" + str(num_graph))
        print(dataset[0])

        # Number of unit cells
        num_unit_cell = len(set(unit_cell))
        print("Number of unit cells:" + str(num_unit_cell))
        
        # # Detect if the training set is too large
        # if num_train_per_unit_cell * num_unit_cell >= num_graph:
        #     print("Training set is larger than the input data!")
        #     return
        
        # Extract training and testing data
        data_train = []
        data_test = []
        unit_cell_train = []
        unit_cell_test = []
        
        uc = list(set(unit_cell))
        
        for i in range(num_unit_cell):

            dataset_tmp = [dataset[k] for k in np.where(np.array(unit_cell) == uc[i])[0]]
            if i == 0:
                print("Temp Dataset Size: " + str(len(dataset_tmp)))
                # print(dataset_tmp)
            random.shuffle(dataset_tmp)
            

            # What is num_per_unit_cell for? is it the number of identical unit cells
            data_train_tmp = dataset_tmp[:num_per_unit_cell]
            data_train.append(data_train_tmp)
            unit_cell_tmp_train = uc[i] * num_per_unit_cell
            unit_cell_train.append(unit_cell_tmp_train)
            
            data_test_tmp = dataset_tmp[num_per_unit_cell:num_per_unit_cell*2]
            data_test.append(data_test_tmp)
            unit_cell_tmp_test = uc[i] * num_per_unit_cell
            unit_cell_test.append(unit_cell_tmp_test)
            
        # Extract shared data
        # Randomly samples n graphs from the training data, n = batch_share
        adj_share = []
        for i in range(num_unit_cell):
            adj_share_tmp = random.sample(data_train[i], self.batch_share) #####
            adj_share.append(self.makeAdj(adj_share_tmp, self.num_pad))
        
        print(len(adj_share))
        print(adj_share[0].shape)

        # Make batches for training data
        batch_remain = self.batch_size - self.batch_share
        adj_train = []
        unit_cell_identifier_train = []

        print(num_unit_cell)
        print(num_per_unit_cell // self.batch_size)


        # adj_train variable, takes the graph and applies zero padding
        # Iterates through each unit cell
        for i in range(num_unit_cell):
            data_train_unit_cell_tmp = data_train[i]
            print(len(data_train_unit_cell_tmp))
            # For each unit cell, makes batches by using n samples and m shared samples, where n = batch_size - m, and m = share size
            for j in range(num_per_unit_cell // self.batch_size):
                adj_train_unit_cell_tmp = torch.cat((adj_share[i], self.makeAdj(data_train_unit_cell_tmp[j * batch_remain:(j+1) * batch_remain], self.num_pad)), dim = 0)
                adj_train.append(adj_train_unit_cell_tmp)
                unit_cell_identifier_train.append(i)
        print(len(unit_cell_identifier_train))
        
        # Make batches for testing data
        adj_test = []
        unit_cell_identifier_test = []

        print("Num")
        print(num_unit_cell)
        for i in range(num_unit_cell):
            data_test_unit_cell_tmp = data_test[i]
            for j in range(num_per_unit_cell // self.batch_size):
                adj_test_unit_cell_tmp = self.makeAdj(data_test_unit_cell_tmp[j * self.batch_size:(j+1) * self.batch_size], self.num_pad)
                adj_test.append(adj_test_unit_cell_tmp)
                unit_cell_identifier_test.append(i)
        
        # Finalize batched training and testing data
        num_graphs_each_unit_cell = int(len(unit_cell_identifier_train) / num_unit_cell)
        adj_train_output = []
        adj_test_output = []
        for i in range(num_graphs_each_unit_cell):
            adj_unit_cell_train = []
            adj_unit_cell_test = []
            for j in range(num_unit_cell):
                adj_unit_cell_train.append(adj_train[j * num_graphs_each_unit_cell + i])
                adj_unit_cell_test.append(adj_train[j * num_graphs_each_unit_cell + i])
            adj_train_output.append(adj_unit_cell_train)
            adj_test_output.append(adj_unit_cell_test)
            
        # Finalize batched test data
        # num_batch_test = int(len(unit_cell_identifier_train) / num_unit_cell)
        # adj_test_output = []
        # for i in range(num_batch_test):
        #     adj_test_tmp = []
        #     for j in range(num_unit_cell):
        #         adj_test_tmp.append(self.makeAdj(graphs = data_test[j * num_batch_test + i], num_pad = self.num_pad))
        #     adj_test_output.append(adj_test_tmp)
        
        return adj_train_output, adj_test_output

In [26]:
graph_loader = Make_batch_data(num_pad = max_num_nodes, batch_size = batch_size, batch_share = batch_share)
graph_train, graph_test = graph_loader.makeTrain(dataset = graphs_whole[:270*20], unit_cell = unit_cell_whole[:270*20], num_per_unit_cell = 50)

Number of graphs:5400
Graph with 8 nodes and 14 edges
Number of unit cells:20
Temp Dataset Size: 270
20
torch.Size([20, 432, 432])
20
0
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
50
0
Num
20


In [18]:
import pandas as pd
x = pd.read_pickle("ZeoGraphs.p")

In [20]:
graph_loader = Make_batch_data(num_pad = max_num_nodes, batch_size = batch_size, batch_share = batch_share)
padded = graph_loader.makeAdj(x[:100*270], 432)

In [23]:
padded.shape

torch.Size([27000, 432, 432])

In [24]:
# max_nodes = 0
# for i in x:
#     if max_nodes < i.number_of_nodes():
#         max_nodes = i.number_of_nodes()
#         y = i
#         print(max_nodes)
#         print(i)
# max_nodes

8
Graph with 8 nodes and 14 edges
16
Graph with 16 nodes and 28 edges
24
Graph with 24 nodes and 42 edges
32
Graph with 32 nodes and 64 edges
48
Graph with 48 nodes and 96 edges
72
Graph with 72 nodes and 144 edges
96
Graph with 96 nodes and 192 edges
144
Graph with 144 nodes and 288 edges
216
Graph with 216 nodes and 432 edges
252
Graph with 252 nodes and 504 edges
378
Graph with 378 nodes and 756 edges
432
Graph with 432 nodes and 864 edges


432

In [27]:
print(i)

Graph with 378 nodes and 756 edges
